In [1]:
import pandas as pd
import numpy as np

In [6]:
news_header = ["news_id", "category", "sub_category","title","abstract", "url", "title_entities", "abstract_entities"]
behaviors_header = ["id", "user_id", "time","history","impressions"]
news = pd.read_csv("news.tsv", sep="\t", names=news_header)
news = news.drop(["category","sub_category","url","title_entities","abstract_entities", "abstract", "title"], axis=1)

unique_users = pd.read_csv("behaviors.tsv", sep="\t", names=behaviors_header)
unique_users = unique_users.drop_duplicates(subset = "user_id")
unique_users = unique_users[unique_users['history'].notna()]
unique_users = unique_users.drop(["id", "time", "impressions"], axis=1)
unique_users["history"] = unique_users["history"].str.split(" ")
unique_users = unique_users.sample(frac = 0.001)

In [8]:
joined = unique_users.merge(news, how='cross')

In [10]:
def score(row):
    if row["news_id"] in row["history"]:
        return 1
    return 0

joined["score"] = joined.apply(score, axis=1)

In [11]:
from scipy import sparse
R_df = sparse.csr_matrix(joined.pivot_table(index = 'user_id', columns ='news_id', values = 'score').to_numpy())

In [12]:
from implicit.als import AlternatingLeastSquares
model = AlternatingLeastSquares(factors=40, regularization=0.01, alpha=1.0, iterations= 200)
model.fit(R_df)

  0%|          | 0/200 [00:00<?, ?it/s]

In [13]:
model.recommend(0, R_df[0])

(array([27872,  9923, 45715,  7760,  3408,  2680, 49794, 31231, 11244,
         3737]),
 array([0.02343327, 0.02284366, 0.02104516, 0.00227207, 0.00151783,
        0.00151783, 0.00151782, 0.00133261, 0.00119897, 0.00101701],
       dtype=float32))